In [1]:
from flask import Flask, request
from flask_cachecontrol import cache_for
import requests

html = ("""
<html>
    <head>
      <title>Weather Forecase</title>
    </head>
    <body>
      <h1>Expect Weather Forecast!!</h1>
      <form action="/showWeather" method="GET">
          <input type="text" name="city" placeholder="city"></input>
          <button>weather</button>
      </form>
    </body>
</html>
""")

app = Flask(__name__)
@app.route("/")
def index():
    return html

In [2]:
@app.route("/showWeather")
@cache_for(hours=3)
def weather():
    ###get city name
    city = request.args.get("city")
    
    ###valid the input
    API_KEY = "eb36c39fd502775c077267afa9ab2867"
    url_loc = f"http://api.openweathermap.org/geo/1.0/direct?q={city},KOR&limit=1&appid={API_KEY}"
    
    try:
        loc = requests.get(url_loc)
    except:
        raise RuntimeError(f"{city} is unvalid.1")
        
    loc = loc.json()
    if not (bool(loc)):
        raise RuntimeError(f"{city} is unvalid.2")
    
    lat = loc[0]['lat']
    lon = loc[0]['lon']
    
    ###fetch
    url = "http://localhost:5001/weather"
    params = {
        'city':city,
        'lat': lat,
        'lon': lon
    }
    res = requests.get(url, params=params)
    if not (res.status_code in range(200, 400)):
        raise RuntimeError(f"something's wrong!")
    res = res.json()
    
    re_html = (f"""
        <html>
            <head>
              <title>Weather Forecast</title>
            </head>
            <body>
              <h1>Weather Forecasting</h1>
              <p>Todya's feels_like temperature: </p>
              <p> Morning : {res['daily'][0]['feels_like']['morn']}</p>
              <p> Day : {res['daily'][0]['feels_like']['day']}</p>
              <p> Night : {res['daily'][0]['feels_like']['night']}</p>
            </body>
        </html>
    """)
    
    return re_html
    
if __name__ == '__main__':
    app.run(host="localhost", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Oct/2023 10:35:05] "GET /showWeather?city=daegu HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2023 10:39:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2023 10:39:22] "GET /showWeather?city=jejudo HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2023 10:40:23] "GET /showWeather?city=seoul HTTP/1.1" 200 -
